# Neural Networks

Neural networks work by connecting together a sequence of "layers" (nodes connected to one another).
In this notebook we'll work with the `Dense` Layer, where every every node from the previous layer is connected to every node in the subsequent layer.

We'll use the [Keras](https://keras.io/) library

When we use Neural Networks, they will train more effectively if we scale the label column (SalePrice) to lie between 0 and 1. We 
then transform it back to its original scale.

In [7]:
from utils import evaluate_model
from keras.models import Sequential
from keras.layers import Dense, Activation
import sklearn
import pandas as pd
import sklearn.preprocessing

In [8]:
def encode_data(data,scaler = None):
    data = data.copy()
    data = data[['FullBath','BedroomAbvGr']]
    data['BedBath'] = data['FullBath'] * data['BedroomAbvGr']
        
    if(not scaler):
        scaler = sklearn.preprocessing.MinMaxScaler()
        scaler.fit(data)
    pd.DataFrame(scaler.transform(data), columns = ['FullBath','BedroomAbvGr','BedBath'])
    return(data,scaler)
    


def train_neural_network_model(training_set):
    training_features, scaler = encode_data(training_set)
    
    
    # 3 Dense layers and an output layer
    predictor = model = Sequential([
    Dense(32, input_shape=(3,)),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(1),
    Activation('relu')
    ])
    
    predictor.compile(optimizer='rmsprop',
              loss='mae')
    
    label_scaler = sklearn.preprocessing.MinMaxScaler()
    scaled_labels = label_scaler.fit_transform(training_set[['SalePrice']])
    predictor.fit(training_features,scaled_labels, epochs = 50)
    
    def neural_network_model(input_data):
        input_features,_ = encode_data(input_data,scaler)
        scaled_predictions = predictor.predict(input_features)
        
        # Scale prediction back to its original scale
        predictions = pd.DataFrame(label_scaler.inverse_transform(scaled_predictions),
                                   columns = ["SalePrice"])
        
        return(predictions['SalePrice'])
    return(neural_network_model)

In [10]:
training_set = pd.read_csv("../data/housing_price_data/training_data.csv")
neural_network_model = train_neural_network_model(training_set)
evaluate_model(neural_network_model)

Epoch 1/50
1168/1168 [==============================] - 0s 222us/step - loss: 0.1033
Epoch 2/50
1168/1168 [==============================] - 0s 31us/step - loss: 0.0840
Epoch 3/50
1168/1168 [==============================] - 0s 30us/step - loss: 0.0803
Epoch 4/50
1168/1168 [==============================] - 0s 34us/step - loss: 0.0769
Epoch 5/50
1168/1168 [==============================] - 0s 33us/step - loss: 0.0735
Epoch 6/50
1168/1168 [==============================] - 0s 33us/step - loss: 0.0732
Epoch 7/50
1168/1168 [==============================] - 0s 33us/step - loss: 0.0714
Epoch 8/50
1168/1168 [==============================] - 0s 31us/step - loss: 0.0682
Epoch 9/50
1168/1168 [==============================] - 0s 34us/step - loss: 0.0673
Epoch 10/50
1168/1168 [==============================] - 0s 32us/step - loss: 0.0659
Epoch 11/50
1168/1168 [==============================] - 0s 32us/step - loss: 0.0660
Epoch 12/50
1168/1168 [==============================] - 0s 32us/step - l

45859.436001712325

# Exercise

1. Play around with different `Activation` functions. Options include 'tanh', and 'sigmoid'
2. Experiment with adding different numbers of nodes and layers.